# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [81]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [82]:
#pull csv from weatherpy
path = "../WeatherPY/city_summary_df.csv"
city_data = pd.read_csv(path)

city_data.head()

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime
0,0,ujiie,JP,1849367,36.68,139.97,48.20,93,75,13.87,1580179815
1,1,puerto madryn,AR,3840092,-42.77,-65.04,77.14,17,10,15.21,1580179816
2,2,antofagasta,CL,3899539,-23.65,-70.40,72.00,88,90,8.05,1580179816
3,3,lompoc,US,5367788,34.64,-120.46,60.01,93,1,11.41,1580179200
4,4,port elizabeth,ZA,964420,-33.92,25.57,57.20,71,33,4.70,1580179802


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [83]:
#configure gmaps
gmaps.configure(api_key=api_key)


In [84]:
locations = city_data[["City Lat", "City Long"]]
weights = city_data['Humidity %']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
#Narrow down the DataFrame to find your ideal weather condition. For example:

#  * A max temperature lower than 90 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

city_name = []


for index, row in city_data.iterrows():
    if row["Cloudiness %"] == 0:
        if row ["Windspeed (mph)"] < 10:
            if row["Temps (F)"] >70 and row["Temps (F)"]<90:
                city_name.append(row)

city_weather_df = pd.DataFrame(city_name)

city_weather_df.to_csv("city_perfect_weather.csv")
city_weather_df.head()
    

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime
40,40,omboue,GA,2396853,-1.57,9.26,75.36,95,0,0.49,1580179242
135,135,general roca,AR,3855065,-39.03,-67.58,73.40,53,0,6.22,1580179902
254,254,beloha,MG,1067565,-25.17,45.05,70.63,94,0,4.36,1580179979
270,270,jardim,BR,3460087,-21.48,-56.14,75.20,65,0,4.47,1580179989
290,290,ahipara,NZ,2194098,-35.17,173.17,81.00,71,0,1.99,1580179722


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
#* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

In [87]:
#base G-Places URL
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="

input = "Hotel"
radius = "5000"

In [88]:
hotel_df = city_weather_df

hotel_df["Hotel Name"] = ''

hotel_df.head()

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime,Hotel Name
40,40,omboue,GA,2396853,-1.57,9.26,75.36,95,0,0.49,1580179242,
135,135,general roca,AR,3855065,-39.03,-67.58,73.40,53,0,6.22,1580179902,
254,254,beloha,MG,1067565,-25.17,45.05,70.63,94,0,4.36,1580179979,
270,270,jardim,BR,3460087,-21.48,-56.14,75.20,65,0,4.47,1580179989,
290,290,ahipara,NZ,2194098,-35.17,173.17,81.00,71,0,1.99,1580179722,


In [89]:
#https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=mongolian%20grill&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:2000@47.6918452,-122.2226413&key=YOUR_API_KEY

In [90]:
#loop through narrowed down cities and find hotels
#hotels = []

for index, row in hotel_df.iterrows():
    latitude = row["City Lat"]
    longitude = row["City Long"]

    target_url = (f"{base_url}{input}&key={api_key}&inputtype=textquery&fields=name,formatted_address&locationbias=circle:{radius}@{latitude},{longitude}")
    
    response = requests.get(target_url).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = response["candidates"][0]["name"]

    except:
        print(f"There are no hotels here!")
        pass

hotel_df.to_csv('hotel_df.csv') 
hotel_df.head(20)

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime,Hotel Name
40,40,omboue,GA,2396853,-1.57,9.26,75.36,95,0,0.49,1580179242,Hotel Olako
135,135,general roca,AR,3855065,-39.03,-67.58,73.40,53,0,6.22,1580179902,Hotel y Casino del Río General Roca
254,254,beloha,MG,1067565,-25.17,45.05,70.63,94,0,4.36,1580179979,Loews Atlanta Hotel
270,270,jardim,BR,3460087,-21.48,-56.14,75.20,65,0,4.47,1580179989,Vitória Hotel
290,290,ahipara,NZ,2194098,-35.17,173.17,81.00,71,0,1.99,1580179722,The Huts
303,303,campo verde,BR,3467673,-20.42,-54.07,75.20,88,0,3.36,1580180011,Hotel Thalis
346,346,cabo rojo,PR,4562996,18.09,-67.15,78.01,77,0,4.45,1580180037,Vistas De La Bahia
396,396,puerto colombia,CO,3671497,10.99,-74.95,80.60,78,0,9.17,1580180069,IMAGINE BEACH HOTEL
399,399,ampanihy,MG,1078553,-24.70,44.75,71.85,93,0,4.43,1580180071,Angora Hotel
482,482,nioro,GM,2412408,13.35,-15.75,71.55,11,0,0.81,1580180122,Tendaba camp


In [91]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_weather_df.iterrows()]
locations = hotel_df[["City Lat", "City Long"]]

In [79]:
# Add marker layer ontop of heat map
locations = hotel_df[["City Lat", "City Long"]]
weights = hotel_df['Humidity %']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>